In [1]:
import glob, numpy as np, sentencepiece as spm, random, json, os

In [8]:
import pathlib, glob, numpy as np, sentencepiece as spm, random

proj_root   = pathlib.Path.cwd().parent   
tokens_root = proj_root / "storage" / "tokens"

train_shards = glob.glob(str(tokens_root / "train" / "tinystories_nano" / "tokens_*.npz"))
val_shards   = glob.glob(str(tokens_root / "val"   / "tinystories_nano" / "tokens_*.npz"))

print(f"train shards = {len(train_shards)}, val shards = {len(val_shards)}")
arr = np.load(random.choice(train_shards))["input_ids"]
print("shape", arr.shape, "dtype", arr.dtype)

sp = spm.SentencePieceProcessor(model_file=str(tokens_root / "tokenizer" / "tokenizer.model"))
print(sp.decode(arr[0][:100].tolist()))

train shards = 2, val shards = 1
shape (10000, 512) dtype uint16
saw a small bird named Tim. Max said, "Tim, can you help me open this sack?" Tim said, "Yes, I can help you." Tim could whistle very loud. Tim whistled loud to call his bird friends. Many birds came to help Max. They all pulled the sack open. Inside the sack, they found lots of yummy treats. Max and the birds were very happy. They shared the treats and had a fun day at the park.<|endoftext|> One day, a boy named


In [9]:
import torch, glob, itertools

class NPZChunks(torch.utils.data.IterableDataset):
    def __init__(self, pattern, shuffle=True):
        self.files   = glob.glob(pattern, recursive=True)
        self.shuffle = shuffle

    def __iter__(self):
        files = self.files.copy()
        if self.shuffle: random.shuffle(files)
        for fname in files:
            data = np.load(fname)["input_ids"]          # (N, 512) uint16
            idxs = range(len(data))
            if self.shuffle: idxs = random.sample(idxs, len(idxs))
            for i in idxs:
                yield torch.from_numpy(data[i]).long()


In [25]:
from torch.utils.data import DataLoader

proj_root = pathlib.Path.cwd().parent   
pattern = str(proj_root / "storage/tokens/train/tinystories_nano/*.npz")
train_dl = DataLoader(
    NPZChunks(pattern),
    batch_size=8, num_workers=1, pin_memory=True, drop_last=True)

In [26]:
next(iter(train_dl))

tensor([[  184,    13,    78,  ..., 29975,   104,    92],
        [  402, 29982, 29955,  ...,   272,    31,   137],
        [   96,   519,   225,  ...,   172,    86,    31],
        ...,
        [  117,    11,   438,  ...,  1846,    66,    11],
        [  224,    86,  4072,  ...,  1502,   114,   205],
        [ 1263,    13,  2181,  ..., 29967,    59,  5255]])

In [29]:
import time, torch
warmup_batches = 10
measure_batches = 100
tok_per_batch = 8 * 512                 # batch_size * seq_len

t0 = time.time()
for i, batch in enumerate(train_dl):
    if i == warmup_batches:
        t0 = time.time()                # start timing after warm-up
    if i == warmup_batches + measure_batches:
        break

elapsed = time.time() - t0
print(f"DataLoader: {measure_batches * tok_per_batch / elapsed:,.0f} tokens/s")

DataLoader: 7,649,913 tokens/s
